# KKBox Music Recommendation
- 문제 정의 : 사용자의 과거 청취 데이터를 기반으로 해당 노래 재청취 여부 예측(Binary Classification)
- 데이터 셋
    - members : 유저에 대한 데이터(나이, 도시, 성별, 구독 경로 등)
    - songs : 노래에 대한 데이터(길이, 장르, 작곡가, 작사가 등)
    - songs_extra_info : 노래에 대한 추가적인 데이터(ISRC 코드)
    - train : 유저, 노래, 어플 관련 데이터(노래가 재생된 탭, 첫 음악 재생 위치 등)
    - test : 상동

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# seaborn의 그래프 스타일 지정
#sns.set_style('ticks')

In [ ]:
# hide warning message
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 데이터 셋 로드
# 각 변수별 적절한 데이터 타입으로 미리 변환.

train = pd.read_csv('../input/train.csv', dtype={'source_system_tab':'category',
                                          'source_screen_name':'category',
                                          'source_type':'category',
                                          'target':np.uint8,
                                          'song_id':'category'})

In [ ]:
members = pd.read_csv('../input/members.csv', dtype={'city':'category'})

In [ ]:
members.rename(index=str, columns = {'bd':'age'}, inplace=True)

In [ ]:
members.isnull().sum()

## city, age, gender 변수의 결측치
- city는 가입시점 기준 유저의 거주 도시이고 `age`는 나이, `gender`는 성별을 의미한다.
- Raw data에서는 `gender` 변수에만 결측치가 있었다.
- 하지만, 1번 도시 거주자와 나이가 0세인 유저가 `gender`의 결측치와 동시에 함께 등장한다는 것을 발견했다.
- 또한, KKBox 어플을 다운받아 확인한 결과, 거주 도시, 나이, 성별과 같은 정보를 입력하지 않고 건너뛰는 것이 가능했다.
- 이를 통해 유저가 해당 정보를 입력하지 않고 건너뛸 경우 `city`는 1로, 'age'는 0으로, `gender`는 결측치로 입력된다고 판단했다.
- 또, 나이가 70세를 초과하는 경우도 결측치로 판단했다.
- 아래는 이를 검증하는 과정이다.

In [ ]:
## City
# 1번 도시 거주 유저 카운트 확인
city_1 = members['city'][members['city'] == '1'].count()
city_null_prop = (city_1 / len(members['city'])) * 100

print('* City')
print('- 1번 도시에 거주하는 유저 수 : {:,}명'.format(city_1))
print('- 전체 행 개수에서 1번 도시 거주 유저가 차지하는 비율 : {}%'.format(city_null_prop.round(2)))

## Age
# 나이가 0세 이하인 유저 수 확인
below_0 = members['age'][members['age'] <= 0].count()

# 나이가 70세 이상인 유저 수 확인
above_70 = members['age'][members['age'] >= 70].count()

# 전체 행 개수에서 결측치가 차지하는 비율
age_null_prop = ((below_0 + above_70) / len(members['age'])) * 100

print('\n* Age')
print('- 0세 이하 또는 70세 이상 : {:,}명'.format(below_0 + above_70))
print('- 전체 행 개수에서 0세 이하 또는 70세 이상 유저의 비율 : {}%'.format(age_null_prop.round(2)))

## Gender
# 성별의 결측치 수 확인
gender_null = members['gender'].isnull().sum()
gender_null_prop = (gender_null / len(members['gender'])) * 100

print('\n* Gender')
print('- 성별의 결측치 수 : {:,}명'.format(gender_null))
print('- 전체 행 개수에서 결측치가 차지하는 비율 : {}%'.format(gender_null_prop.round(2)))

- 결측치의 개수와 전체 행에서 결측치가 차지하는 비율을 확인한 결과, 세 변수 모두 거의 비슷했다.

In [ ]:
# city 변수의 1번 도시를 결측 처리
members['city'].replace('1', np.NaN, inplace=True)

# age 변수의 이상치를 결측 처리
members['age'][(members['age'] <= 0) | (members['age'] > 70)] = np.NaN

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(9, 7))

city_null = members['city'].isnull().sum()
age_null = members['age'].isnull().sum()
gender_null = members['gender'].isnull().sum()

var_rows = (city_null, age_null, gender_null)
total_rows = (len(members) - city_null, len(members) - age_null, len(members) - gender_null)
N = 3
ind = np.arange(N)
width = 0.35

p1 = plt.bar(ind, var_rows, width)
p2 = plt.bar(ind, total_rows, width, bottom = var_rows)


plt.title('proportion of missing value in three variables', fontsize=18, fontweight=700)
plt.xticks(ind, ('city', 'age', 'gender'), fontsize=16, fontweight=500)
plt.ylabel('count of rows', fontsize=16, fontweight=700)
plt.xlabel('columns', fontsize=16, fontweight=700)
plt.legend((p1[0], p2[0]), ('missing values', 'total rows'), loc=0)


#df.plot.bar(rot=0, subplots=True)
#axes = members.plot.bar(rot=0, subplots=True)
#axes[1].legend(loc=2)  # doctest: +SKIP

In [ ]:
## 결측치가 동시에 발생하는지 확인

# city와 gender가 동시에 결측치가 발생하는 경우
city_gender = members[(members['city'].isnull()) & (members['gender'].isnull())]

# city와 age가 동시에 결측치가 발생하는 경우
city_age = members[(members['city'].isnull()) & (members['age'].isnull())]

# age와 gender가 동시에 결측치가 발생하는 경우
age_gender = members[(members['age'].isnull()) & (members['gender'].isnull())]

# city, age, gender가 동시에 결측치가 발생하는 경우
city_age_gender = members[(members['city'].isnull()) & (members['age'].isnull()) & (members['gender'].isnull())]

print('- city와 gender가 동시에 결측치가 발생하는 경우 : {:,}건'.format(len(city_gender)))
print('- city와 age가 동시에 결측치가 발생하는 경우 : {:,}건'.format(len(city_age)))
print('- age와 gender가 동시에 결측치가 발생하는 경우 : {:,}건'.format(len(age_gender)))
print('- city, age, gender가 동시에 결측치가 발생하는 경우 : {:,}건'.format(len(city_age_gender)))

- 세 변수가 동시에 등장할 수 있는 모든 경우의 수를 확인한 결과, 거의 비슷했다.
- 정확히 일치하지 않는 이유는 3개의 값 중 한두개의 값만 입력한 유저가 있기 때문일 것이다.
- 이를 토대로 `city` 변수에서 값이 1인 경우와 `age`가 0세 이하 70세 초과인 경우도 모두 결측치로 변환했다.

## Q. 많이 청취된 곡은 재청취율도 높을까?

In [ ]:
song_grouped = pd.DataFrame(train.groupby('song_id')\
                            .agg({'song_id':'count','target':'mean'})\
                            .rename(index=str, columns={'song_id':'count','target':'mean_target'})\
                            .reset_index()\
                            .groupby('count')\
                            .agg({'song_id':'count', 'mean_target':'mean'})\
                            .reset_index()\
                            .rename(index=str, columns={'count':'occurrence', 'song_id':'number of songs', 'mean_target':'avg_target'}))
print(song_grouped.head())
print(song_grouped.tail())

- '많이 청취된 곡은 재청취율도 높을까?' 이 물음에 대해 답하기 위해 위와 같이 두 번의 그룹화를 했다.
- occurence는 곡의 등장횟수를 의미하고 number of songs는 그만큼 등장한 곡의 수, avg_target은 해당 곡들의 평균 재청취율을 의미한다.
- 예를 들어 첫 번째 행의 경우 train 테이블에서 딱 1회 등장한 곡이 166,766개이며 이 곡들의 평균 재청취율은 37.7%라는 것을 의미한다.
- 위의 head()와 tail()의 결과를 보면, 곡의 등장횟수가 많을 수록 재청취율도 높아진다는 것을 대략적으로 알 수 있지만, 더 정확히 알기 위해 그래프를 그려보기로 한다.

In [ ]:
plt.figure(figsize=(9, 7))
sns.lineplot( x='occurrence', y='avg_target', data=song_grouped)

In [ ]:
# 재생횟수와 재청취율의 상관관계

np.corrcoef(song_grouped['occurrence'], song_grouped['avg_target']) # 66%

## Q. 노래를 많이 듣는 유저는 재청취율이 높을까?

In [ ]:
user_grouped = pd.DataFrame(train.groupby('msno')\
                            .agg({'msno':'count','target':'mean'})\
                            .rename(index=str, columns={'msno':'count','target':'mean_target'})\
                            .reset_index()\
                            .groupby('count')\
                            .agg({'msno':'count', 'mean_target':'mean'})\
                            .reset_index()\
                            .rename(index=str, columns={'count':'occurrence', 'msno':'number of users', 'mean_target':'avg_target'}))
print(user_grouped.head())
print(user_grouped.tail())

In [ ]:
np.corrcoef(user_grouped['occurrence'], user_grouped['avg_target'])

In [ ]:
top_users = pd.DataFrame(train.groupby('msno')\
                         .agg({'msno':'count','target':'mean'})\
                         .rename(index=str, columns={'msno':'count','target':'mean_target'})\
                         .reset_index()\
                         .sort_values('count', ascending=False))

In [ ]:
top_users.head()

In [ ]:
top_users2 = pd.DataFrame(train.groupby('msno')\
                         .agg({'msno':'count','target':'mean'})\
                         .rename(index=str, columns={'msno':'count','target':'mean_target'})\
                         .reset_index()\
                         .sort_values('mean_target', ascending=False))

In [ ]:
top_users2.head(100)